## Imports

In [ ]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import pickle
from sklearn.decomposition import PCA


# webscraping
import requests
from bs4 import BeautifulSoup

# stocks
import yfinance as yf

# fama french
import statsmodels.api as sm
from statsmodels import regression

# markowitz
from TracyWidom import TracyWidom
import cvxopt as opt
from cvxopt import blas, solvers
import cufflinks
import mplfinance as mpf
import plotly.tools as tls
from plotly.graph_objs import *
solvers.options['show_progress'] = False

# ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# import sys
# !{sys.executable} -m pip install mplfinance

## Basic Goal

Plan: 
1. Data gathering
      - Randomly select 30 stocks from the S&P 500 
      - Get data for each of the stocks from the past 3 years using yfinance.  
      - Get the daily Fama-French factors from the Kenneth French website.
2. Implement the PCA Markowitz portfolio optimization
      - PCA on normalized returns
      - Check if PC1 is significant using Tracy-Widow
      - Get the portfolio that corresponds to PC1
3. Implement the Fama-French three-factor model
      - For each stock, run the standard time series regression for the Fama-French model. 
      - Get the covariance matrix from the residuals 
4. Compare the 2 portfolios against the efficient frontier

### Data Gathering

In [5]:
# getting the stocks
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    )
}

response = requests.get(
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    headers=headers
)
response.raise_for_status()
tables = pd.read_html(response.text)

if len(tables) > 0:
    stocks_df = tables[0]

# randomly selecting 30 stocks
random_stocks = stocks_df['Symbol'].sample(n=30, random_state=42)

In [6]:
# getting closing prices for the 30 stocks with batching
start_date = '2022-08-31'
end_date = '2025-08-31'

# def download_stocks_in_batches(tickers, batch_size=5, delay=1):
#     """
#     Download stock data in batches to avoid rate limiting
#     """
#     all_data = {}
    
#     for i in range(0, len(tickers), batch_size):
#         batch = tickers[i:i + batch_size]
#         print(f"Downloading batch {i//batch_size + 1}: {batch}")
        
#         try:
#             # Download the batch
#             batch_data = yf.download(
#                 batch,
#                 start=start_date,
#                 end=end_date,
#                 progress=False
#             )
            
#             # Extract closing prices for this batch
#             if not batch_data.empty and 'Close' in batch_data.columns:
#                 closes = batch_data['Close']
#                 # Handle single ticker case (returns Series instead of DataFrame)
#                 if isinstance(closes, pd.Series):
#                     all_data[batch[0]] = closes
#                 else:
#                     for ticker in closes.columns:
#                         all_data[ticker] = closes[ticker]
#                 print(f"Successfully downloaded {len(batch)} stocks")
#             else:
#                 print(f"No data returned for batch: {batch}")
            
#         except Exception as e:
#             print(f"Error downloading batch {batch}: {e}")
        
#         # Add delay between batches to avoid rate limiting
#         if i + batch_size < len(tickers):
#             print(f"Waiting {delay} seconds before next batch...")
#             time.sleep(delay)
    
#     # Combine all data into a single DataFrame
#     if all_data:
#         return pd.DataFrame(all_data)
#     else:
#         return pd.DataFrame()

# # Download in batches of 5 stocks with 1-second delay
# closing_df = download_stocks_in_batches(
#     random_stocks.tolist(), 
#     batch_size=5, 
#     delay=15
# )

This code above was generated with ChatGPT.

In [7]:
# if not closing_df.empty:
#     closing_df.to_pickle('closing prices.pkl')

# closing_df.head(5)

In [8]:
# opening pkl file
filename = r'closing prices.pkl'
with open(filename, 'rb') as f: 
    closing_df = pickle.load(f)

# getting simple returns
simple_df = closing_df / closing_df.shift()
simple_df.dropna(how='all', inplace=True)

# getting log returns
log_df = np.log(simple_df)
log_df.round(2)

,BR,CINF,DHI,K,LIN,BA,GLW,IDXX,LHX,O,A,CAH,FITB,MTCH,PNC,AMP,MAA,RJF,SBAC,XYL,BLDR,HCA,HSIC,WMB,WTW,CB,NWS,ROP,UNH,VTRS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-01,0.00,0.01,0.00,0.01,-0.01,-0.04,-0.01,0.01,0.00,0.00,0.01,0.01,-0.01,-0.02,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,-0.01,-0.01,0.00,0.02,0.00,0.01,0.01,0.00
2022-09-02,-0.03,-0.01,0.00,-0.01,-0.01,-0.01,-0.02,-0.02,-0.00,-0.01,-0.01,-0.00,-0.01,-0.00,-0.01,-0.00,-0.02,-0.01,-0.03,-0.01,-0.00,0.00,-0.02,0.00,-0.01,-0.01,0.00,-0.01,-0.01,-0.01
2022-09-06,0.01,0.00,-0.02,-0.01,-0.00,0.00,-0.01,-0.01,0.01,0.00,0.01,-0.02,-0.01,-0.02,-0.01,0.00,0.02,-0.00,0.00,0.01,-0.02,0.00,-0.00,-0.02,0.00,-0.00,-0.03,-0.01,0.00,-0.00
2022-09-07,0.01,0.02,0.02,0.01,0.03,0.02,0.01,0.04,0.01,0.01,0.02,0.02,0.02,0.07,0.02,0.03,0.02,0.02,0.02,0.04,0.02,0.06,0.02,0.00,0.03,0.02,0.02,0.02,0.01,0.03
2022-09-08,0.01,0.01,0.01,-0.02,-0.01,0.01,0.00,0.03,0.00,-0.03,0.03,-0.00,0.04,0.03,0.02,0.02,0.01,0.02,0.01,0.01,0.00,0.02,0.01,-0.03,0.00,0.01,-0.00,0.00,0.01,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-25,-0.02,-0.01,-0.01,-0.00,-0.01,-0.01,0.02,-0.01,0.00,-0.01,-0.02,-0.01,-0.01,-0.00,-0.00,-0.01,-0.01,0.00,-0.01,-0.01,-0.02,-0.01,-0.02,-0.00,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
2025-08-26,0.00,0.00,-0.00,-0.00,0.01,0.03,0.01,0.00,0.01,-0.00,-0.01,0.01,0.01,0.00,0.01,0.01,0.00,0.01,-0.01,0.00,0.01,-0.00,0.01,0.01,-0.01,-0.01,-0.01,-0.01,-0.01,-0.01
2025-08-27,0.00,0.00,-0.01,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.01,0.00,0.01,0.01,0.00,0.01,0.00,0.01,-0.00,-0.05,0.01,-0.01,0.01,-0.00,0.01,-0.01,0.00,-0.01,0.01,0.01,0.00


In [9]:
# getting fama french factors
ff_data = pd.read_csv(
    'ff_factors.csv', 
    header=2
)
ff_data.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
ff_data = ff_data.iloc[:422]

# getting dates
ff_data['date'] = ff_data['date'].astype(str).str.strip()

for idx in range(len(ff_data['date'])):
    date = ff_data.iloc[idx, 0]
    ff_data.iloc[idx, 0] = date[:4] + '-' + date[-2:]

ff_data['date'] = pd.to_datetime(ff_data['date'])
ff_data.set_index('date', inplace=True)
ff_data = ff_data.loc[start_date:end_date]
ff_data

,Mkt-RF,SMB,HML,RF
date,,,,
2022-09-01,-9.46,-1.66,1.89,0.19
2022-10-01,6.79,-2.02,4.41,0.23
2022-11-01,7.25,-0.24,-0.28,0.29
2022-12-01,-4.33,2.14,2.51,0.33
2023-01-01,6.92,0.98,-1.89,0.35
2023-02-01,-2.67,-0.04,0.69,0.34
2023-03-01,2.27,-3.23,-6.92,0.36
2023-04-01,1.22,-1.75,1.00,0.35
2023-05-01,-1.61,-0.66,-4.76,0.36


### Implement the PCA Markowitz portfolio optimization

In [24]:
returns_df = simple_df.pct_change().dropna()

norm_returns = (returns_df - returns_df.mean()) / returns_df.std()
norm_returns

,BR,CINF,DHI,K,LIN,BA,GLW,IDXX,LHX,O,A,CAH,FITB,MTCH,PNC,AMP,MAA,RJF,SBAC,XYL,BLDR,HCA,HSIC,WMB,WTW,CB,NWS,ROP,UNH,VTRS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-09-02,-1.727142,-0.742467,-0.073869,-0.978541,0.211179,0.969784,-0.339167,-0.763822,-0.166856,-0.795473,-0.482057,-0.658598,0.127349,0.356679,-0.424854,-0.309737,-1.187196,-0.507704,-1.096479,-0.912341,-0.159984,-0.223360,-0.447511,0.825191,-0.666831,-1.245444,-0.011060,-0.968563,-0.804935,-0.530614
2022-09-06,2.043215,0.571015,-0.657904,0.155459,0.122158,0.499809,0.409046,0.344717,0.624028,0.693833,0.643930,-0.823242,-0.254505,-0.531505,0.066084,0.117286,1.769476,0.372424,1.131731,1.059827,-0.327211,0.009781,0.688862,-0.978992,0.633751,0.223701,-1.379492,0.018777,0.573029,0.282577
2022-09-07,0.078249,0.612604,1.127380,1.057040,1.933386,0.617109,0.667510,1.368944,0.159112,0.384122,0.226841,2.126202,1.217523,2.455665,1.036134,1.276105,-0.188206,0.964216,0.724147,0.939378,0.792951,2.244755,0.981322,0.950131,1.309531,1.060436,2.158590,2.021210,0.157975,1.304501
2022-09-08,-0.106986,-0.522757,-0.076313,-1.821949,-2.098169,-0.383640,-0.109007,-0.285140,-0.618065,-2.060936,0.439344,-1.271282,0.476449,-0.997333,-0.029229,-0.471386,-0.214161,-0.266586,-0.661546,-1.265118,-0.408577,-1.525828,-0.205070,-1.426708,-1.097805,-0.531037,-0.958380,-1.238131,0.100072,-1.282285
2022-09-09,0.014320,0.142150,0.100420,1.386721,0.981101,-0.450297,0.476458,-0.131306,0.386960,2.046416,-0.402237,-0.294510,-0.876481,0.587644,-0.446131,-0.202338,-0.364385,-0.307956,-0.070344,-0.098842,0.985593,-0.132906,-0.062157,2.444221,0.377253,-0.441473,1.902472,0.109256,-0.485148,0.864395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-25,-1.341237,-1.262312,-1.964281,-0.259497,-0.559419,-1.262382,0.058178,-0.820905,0.125122,-0.066254,-2.153463,0.450212,-1.817171,-0.457605,-1.711501,-1.381523,-1.035437,-0.670875,-0.569841,-1.428365,-2.294652,-0.313624,-1.579457,0.393670,-0.762384,-0.436060,-1.447994,-1.755655,-0.739933,-0.609328
2025-08-26,1.209924,0.658290,0.196133,0.020543,0.911863,1.603218,-0.249047,0.207227,0.188197,0.617410,0.499717,1.198562,0.828636,0.147712,0.730459,0.875287,0.536936,0.343646,-0.260625,0.537124,0.545129,0.134251,1.090775,0.440749,0.037272,0.159887,0.032654,0.061005,-0.226703,-0.153055
2025-08-27,0.042311,0.028062,-0.069300,0.064111,-0.387889,-1.011651,-0.463999,-0.075674,-0.442119,0.490142,0.385022,0.086653,-0.120954,-0.006561,-0.054806,-0.337543,0.400761,-0.523784,-1.421263,0.178843,-0.302649,0.399103,-0.435690,0.037065,-0.241548,0.691791,0.293646,1.097957,0.875834,0.494938


In [ ]:
# CHAT GPT

# --- Fit PCA on your dataframe ---
pca = PCA().fit(norm_returns)
eigs = pca.explained_variance_        # eigenvalues (variances of each PC)

# --- Get n (samples) and p (features) ---
n, p = log_df.shape

# --- Tracy–Widom scaling (Johnstone 2001) ---
sqrt_n = np.sqrt(n)
sqrt_p = np.sqrt(p)
mu = (sqrt_n + sqrt_p)**2 / n
sigma = (sqrt_n + sqrt_p) * (1/sqrt_n + 1/sqrt_p)**(1/3) / n

# --- Convert sklearn’s (1/(n-1)) scaling to 1/n convention ---
lambda1 = eigs[0] * (n - 1) / n

# --- Compute TW statistic and p-value ---
tw_stat = (lambda1 - mu) / sigma
tw = TracyWidom(beta=1)
pval = 1 - tw.cdf(tw_stat)

print(f"Largest eigenvalue: {lambda1:.5f}")
print(f"Tracy–Widom statistic: {tw_stat:.5f}")
print(f"p-value: {pval:.5e}")


Largest eigenvalue: 10.38672
Tracy–Widom statistic: 338.98111
p-value: 0.00000e+00


In [39]:
pc1 = pd.Series(pca.components_[0], index=norm_returns.columns)
pc1 = pc1/pc1.sum()
pc1_sorted = pc1.sort_values(ascending=False)

pc1_sorted


AMP     0.045439
PNC     0.045038
FITB    0.042291
XYL     0.041963
LIN     0.040284
CINF    0.040181
MAA     0.040013
ROP     0.039853
RJF     0.039435
NWS     0.038300
A       0.038145
BR      0.038024
GLW     0.034971
BLDR    0.034252
WTW     0.033591
IDXX    0.033455
O       0.033290
WMB     0.031878
VTRS    0.031600
DHI     0.031257
CB      0.030782
SBAC    0.030419
MTCH    0.030066
HSIC    0.029940
HCA     0.028890
BA      0.028199
LHX     0.027274
CAH     0.021310
K       0.011473
UNH     0.008387
dtype: float64

### Implement the Fama-French three-factor model

In [8]:
ff_data.head()

,Mkt-RF,SMB,HML,RF
date,,,,
2022-09-01,-9.46,-1.66,1.89,0.19
2022-10-01,6.79,-2.02,4.41,0.23
2022-11-01,7.25,-0.24,-0.28,0.29
2022-12-01,-4.33,2.14,2.51,0.33
2023-01-01,6.92,0.98,-1.89,0.35


In [9]:
closing_df.head()

,BR,CINF,DHI,K,LIN,BA,GLW,IDXX,LHX,O,...,BLDR,HCA,HSIC,WMB,WTW,CB,NWS,ROP,UNH,VTRS
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-31,162.000641,89.028946,69.112213,61.727592,270.903931,160.250000,31.433512,347.619995,212.855453,57.753407,...,58.610001,192.430176,73.410004,29.283257,198.133423,180.763977,16.726734,394.888794,491.874176,8.328449
2022-09-01,162.511734,89.726776,69.287071,62.236763,268.451660,153.660004,31.021360,349.929993,213.249161,57.897182,...,58.759998,193.966721,72.690002,28.896025,198.382477,183.718567,16.755842,397.409668,496.297302,8.345892
2022-09-02,157.561874,88.900414,69.335640,61.693657,267.076385,151.820007,30.361912,343.829987,212.977310,57.254353,...,58.570000,194.530807,71.260002,29.016495,196.140869,182.513779,16.784945,393.260559,489.051605,8.241241
2022-09-06,158.877426,89.267677,67.985435,61.328754,266.335876,152.389999,30.032190,341.730011,215.386414,57.305103,...,57.639999,195.192078,70.980003,28.543217,196.303726,182.083481,16.290131,389.336914,490.311462,8.206357
2022-09-07,160.476883,90.929634,69.053932,62.092487,274.885406,155.949997,30.215370,354.679993,218.554840,57.744949,...,58.630001,206.327332,72.309998,28.646477,201.409637,185.191025,16.590900,399.067383,494.099915,8.476705


In [ ]:
# getting excess return per stock 


### Compare the 2 portfolios against the efficient frontier

## Intermediate Goal

Plan:
1. Implement the RIE Markowitz portfolio optimization
      - Estimate the covariance matrix by doing RIE on the log returns
2. Check which eigenvectors are significant using Marchenco-Pastur and pick one to use for the portfolio (this should be the second largest eigenvalue).
3. Compare its fit to the efficient frontier.

### Implement the RIE Markowitz portfolio optimization

### Check which eigenvectors are significant using Marchenco-Pastur and pick one to use for the portfolio

### Compare its fit to the efficient frontier.

## Advanced Goal

Plan:
As per the study done by Molero-Gonzales et al., (2023), the steps we will follow are the ff:
1. Apply the Fama-French 3 Factor Model on the log returns matrix.
2. Turn the coefficients for each of the factors for each stock in the portfolio into a matrix.
3. Apply RMT.
      - Eigenvalue decomposition
      - Identify the significant eigenvectors using the Tracy-Widom distribution.
4. Create a portfolio out of the significant factors and the residuals.
5. Compare its performance against the matrix in the Basic Goal section to predict stock risk.

### Apply the Fama-French 3 Factor Model on the log returns matrix.

### Turn the coefficients for each of the factors for each stock in the portfolio into a matrix.

### Apply RMT

### Create a portfolio out of the significant factors and the residuals.

### Compare its performance against the matrix in the Basic Goal section to predict stock risk.